In [ ]:
import os
import copy
from collections import defaultdict
from ego4d.internal.colmap.preprocess import (
    ColmapConfig,
    produce_colmap_script,
    get_colmap_data_dir,
)

%matplotlib inline
import matplotlib.pyplot as plt

from matplotlib import rcParams

import cv2
import numpy as np
import pycolmap

# Generate COLMAP inputs + Validate

Pleae generate the configuration as appropriate, and check the visualizations below under "Validate Configuration".

Once you have generated a validation configuration, you may upload this to S3. Please refer to the "Upload Configuration" section.

## Configure + Generate

Please refer to the inline documentation below.

Please set `COLMAP_BIN` and `VRS_BIN`. These should be setup before running the notebook, please refer to the [README](https://github.com/facebookresearch/Ego4d/blob/main/ego4d/internal/colmap/README.md#setup-and-installation).

In [ ]:
COLMAP_BIN = None  # supply your COLMAP_BIN path here
VRS_BIN = None  # supply your path to VRS here

# be sure to use an absolute path
OUTPUT_DIR = "/private/home/miguelmartin/ego4d/ego4d_public/colmap_experiments/"

In [ ]:
config = ColmapConfig(
    # if using a metadata path, please
    # set video_source and take_id to None
    # as this will be derived for you
    # COMMENT/UNCOMMENT BEGIN
    in_metadata_path="s3://ego4d-consortium-sharing/internal/egoexo_pilot/unc/T1/metadata.json",
    in_videos=None,
    video_source=None,
    take_id=None,
    
    # whether we are taking synchronized exo-centric frames
    sync_exo_views=True,
    # where is the walkaround in the VRS file
    aria_walkthrough_start_sec=None,
    aria_walkthrough_end_sec=None,
    # COMMENT/UNCOMMENT END


    # FOR TESTING ONLY: uncomment the above to use specific paths.
    # NOTE: you cannot enable sync_exo_views if you are using specific paths
    # COMMENT/UNCOMMENT BEGIN
#     in_metadata_path=None,
#     video_source="penn",
#     take_id="0303_Violin_2",
#     in_videos={
#         "aria01": "s3://ego4d-penn/data/0303_Violin_2/ego/aria/c2e4b041-4e68-4b75-8338-f8c625429e75.vrs",
#         "cam01": "s3://ego4d-penn/data/0303_Violin_2/exo/gp01/GX010190.MP4",
#         "cam02": "s3://ego4d-penn/data/0303_Violin_2/exo/gp02/GX010175.MP4",
#         "cam03": "s3://ego4d-penn/data/0303_Violin_2/exo/gp03/GX010012.MP4",
#         "cam04": "s3://ego4d-penn/data/0303_Violin_2/exo/gp04/GX010195.MP4",
#         "mobile": "s3://ego4d-penn/data/0303_Violin_2/exo/mobile/GX010020.MP4",
#     },
#     sync_exo_views=False,
#     aria_walkthrough_start_sec=30.0,
#     aria_walkthrough_end_sec=200.0,
    # COMMENT/UNCOMMENT END

    output_dir=OUTPUT_DIR,  # where to save data

    # there are three rot_mode's:
    # - 0 => perform no rotation
    # - 1 => rotate aria to exo
    # - 2 => rotate exo to aria
    rot_mode=1,
    
    # refer to https://colmap.github.io/cameras.html
    camera_model="OPENCV_FISHEYE",
    
    # the inverse of the number of frames per second to sample
    # the mobile walkaround and aria walkaround
    frame_rate=0.25,
    
    # specific mobile frames to use
    # if None then all frames are considered using `frame_rate`
    mobile_frames=None,
    
    # whether to include the aria walkaround
    include_aria=True,
    # if aria_use_sync_info is True then aria_last_walkthrough_sec will be assigned 
    # based off timesync data
    aria_use_sync_info=False,
    # specific frames for the walkaround 
    # if not provided will use all frames in aria will be used (subsampled using `frame_rate`)
    aria_frames=None,
    
    # where to sample the exo videos from
    exo_from_frame=700,
    exo_to_frame=720,
    # specific frames for each exo video
    # if one is provided below, all must be provided and exo_from_frame/exo_to_frame must be None
    exo_frames=None,
    
    # the name of the configuration (a relative path/directory where all frames and config is saved to)
    # if none a name will be automatically constructed from the above configuration
    name=None,
    
    # misc
    aria_fps=30,      # aria is assumed to be (approx) 30fps by default
    exo_fps=None,     # if none, this will be determined from the video file
    mobile_fps=None,  # if none, this will be determined from the video file
    run_colmap=False, # whether we want to run colmap after
    colmap_bin=COLMAP_BIN, # where colmap is located, if None a default will be provided
    vrs_bin=VRS_BIN,       # where vrs is located, if None a default will be provided
    download_video_files=True, # whether we stream from S3 or download videos (changeme if timeout occurs)
    force_download=False, # if download_video_files is True, force_download will force a re-download of video files
)

In [ ]:
config_used = produce_colmap_script(copy.deepcopy(config))

# Validate Configuration

The below section visualizes the exo-centric frames. Please verify that there is no occulusions in each frame. If there is an occulusion, such as a QR code, this may cause issues with SfM (COLMAP).

In [ ]:
root_dir = get_colmap_data_dir(config_used)
frames_dir = os.path.join(root_dir, "frames")
exo_img_paths = {
    x: [
        os.path.join(frames_dir, x, p)
        for p in os.listdir(os.path.join(frames_dir, x))
    ]
    for x in os.listdir(frames_dir)
    if x.startswith("cam")
}

N_frames = max(len(v) for _, v in exo_img_paths.items())
N_cams = len(exo_img_paths)
sorted_is = sorted(exo_img_paths.items(), key=lambda x: x[0])
viz = [
    (cam_i, frame_i, name + f"_frame_{frame_i}", path)
    for cam_i, (name, paths) in enumerate(sorted_is)
    for frame_i, path in enumerate(paths)
]

fig = plt.figure(figsize=(N_cams * 4, N_frames * 4))
ax = fig.subplots(N_cams, N_frames)
plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0.1)

for i, j, name, path in viz:
    x = cv2.imread(path)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    h, w = x.shape[0:2]
    ax[i][j].set_xticks([], [])
    ax[i][j].set_yticks([], [])
    ax[i][j].imshow(x)
    ax[i][j].set_title(name)

# (Optional) Run COLMAP

In [ ]:
# The location is to be determined, please hold off on uploading for now
colmap_dir = get_colmap_data_dir(config_used)

# Please run the run_colmap.sh script
colmap_script = os.path.join(colmap_dir, "run_colmap.sh")
assert os.path.exists(colmap_script)
print("Please run:")
print()
print(f"sh {colmap_script}")

# Upload Configuration

Please upload your configuration to S3

In [ ]:
dir_to_upload = colmap_dir
print(f"Please upload {dir_to_upload} to s3://<bucket>/<path_to_data>/<capture_uid>/colmap/ via:")
print()
print(f"aws s3 sync {dir_to_upload} <s3_path>")

# Validate COLMAP Ouptuts

After COLMAP has finished running, you can validate the outputs of COLMAP by visualizing view transfer of camera centers.

In [ ]:
COLMAP_WORKING_DIR = None
if COLMAP_WORKING_DIR is None:
    print("WARN: using config working dir (check that this is correct)")
    COLMAP_WORKING_DIR = get_colmap_data_dir(config)

print(f"Using: {COLMAP_WORKING_DIR}")

## Quantative

At the moment, we do not have another method to quantify results other than what COLMAP has produced in it's analysis. Please ensure:

1. All cameras are registered
2. There is a significant amount of images in the model
3. The reproject is relatively low (<= 1-2 pixels)
4. There is one model produced

In [ ]:
analysis_file = os.path.join(COLMAP_WORKING_DIR, "analysis_0.txt")

all_models_dir = os.path.join(COLMAP_WORKING_DIR, "colmap_model")
all_models = [os.path.join(all_models_dir, x) for x in os.listdir(all_models_dir)]
if len(all_models) > 1:
    model_dir = all_models[0]
    print(f"Multiple models were generated ({len(models)}) instead of one.")
elif len(all_models) == 0:
    model_dir = None
    print(f"No models produced.")
else:
    model_dir = all_models[0]

frame_dir = os.path.join(COLMAP_WORKING_DIR, "frames")

if os.path.exists(analysis_file):
    !cat $analysis_file
else:
    print("No analysis file present.")

## Visualize (Qualatative)

### Helper Functions

In [ ]:
# https://github.com/colmap/colmap/blob/d6f528ab59fd653966e857f8d0c2203212563631/scripts/python/read_write_model.py#L453
def qvec2rotmat(qvec):
    return np.array([
      [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
       2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
       2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
      [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
       1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
       2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
      [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
       2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
       1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])

In [ ]:
# return NxN array
# row => camera
# col => image
def get_camera_centers(
    camera_names,
    frame_indices,
    cam_exts,
    recon,
    include_mobile_aria_in_exo,
):
    ms = [
        cam_exts[name][frame_idx] 
        if frame_idx is not None 
        else None
        for name, frame_idx in zip(camera_names, frame_indices)
    ]
    N = len(ms)

    ret = []
    for i in range(N):
        row = []

        m = ms[i]
        if m is None:
            ret.append(None)
            continue
        
        c = m["camera_center"]
        for j in range(N):
            if i == j or ms[j] is None:
                row.append(None)
                continue
            if "cam" in camera_names[j] and \
                ("mobile" in camera_names[i] or "aria" in camera_names[i]) and \
                not include_mobile_aria_in_exo:
                print(f"Skipping {camera_names[i]} in {camera_names[j]}")
                row.append(None)
                continue

            mj = ms[j]
            assert mj is not None

            j_cam_id = mj["cam_id"]
            j_img_id = mj["img_id"]
            j_img = recon.images[j_img_id]
            row.append(tuple(
                x
                for x in recon.cameras[j_cam_id].world_to_image(
                    j_img.project(c)
                )
            ))
        ret.append(row)
    return ret

In [ ]:
def draw_all(cam_names, centers, imgs_by_name, thickness=50, color=(255, 0, 0), radius=10, n_cols=2):
    N = len(cam_names)
    assert N % n_cols == 0
    n_rows = N // n_cols
    
    fig = plt.figure(figsize=(36*n_rows, 36*n_cols))
    ax = fig.subplots(n_rows, n_cols)
    plt.subplots_adjust(wspace=0, hspace=0)

    for row in range(n_rows):
        for col in range(n_cols):
            i = (n_rows - 1) * row + col
            name = cam_names[i]
            img_idx, img = copy.deepcopy(imgs_by_name[name])
            assert img_idx == i
            
            cs = [xs[i] if xs is not None else None for xs in centers]
            for c in cs:
                if c is None:
                    continue
                img = cv2.circle(img, tuple(int(x) for x in c), radius=radius, color=color, thickness=thickness)

            ax_i = row
            ax_j = col
            ax_to_use = None
            if n_cols == 1 and n_rows == 1:
                ax_to_use = ax
            elif n_cols == 1:
                ax_to_use = ax[ax_j]
            else:
                ax_to_use = ax[ax_i][ax_j]

            if img is not None:
                ax_to_use.imshow(img)
            ax_to_use.set_title(f"{name}")
            ax_to_use.set_xticks([], [])
            ax_to_use.set_yticks([], [])

### Load

In [ ]:
cameras_db_path = os.path.join(model_dir, "cameras.txt")
images_db_path = os.path.join(model_dir, "images.txt")
points3D_db_path = os.path.join(model_dir, "points3D.txt")

cameras_txt = open(cameras_db_path).readlines()
images_txt = open(images_db_path).readlines()
p3d_txt = open(points3D_db_path).readlines()

In [ ]:
# images
# https://github.com/rawalkhirodkar/ego4dv2/blob/main/tools/calibration/vis_camera_centers.py#L86
image_data = [
    line.strip().split()
    for line in images_txt[4:][0::2]
]

cam_exts = defaultdict(list)
for line in image_data:
    image_path = line[-1]
    camera_name = image_path.split('/')[0]
    image_name = image_path.split('/')[1]
    cam_id = line[-2]
    img_id = int(line[0])
    
    qvec = np.asarray([float(element) for element in line[1:5]]) ## QW, QX, QY, QZ
    translation = np.asarray([float(element) for element in line[5:8]]) ## TX, TY, TZ
    rotmat = qvec2rotmat(qvec=qvec)
    colmap_camera_center = -1*np.dot(rotmat.T, translation) ## -R^t * T

    exts = {
        'camera_center': colmap_camera_center,
        'image_name': image_name,
        'image_path': os.path.join(frame_dir, camera_name, image_name),
        'qvec': qvec,
        'translation': translation,
        'rotmat': rotmat,
        'cam_id': int(cam_id),
        'img_id': int(img_id),
    }
    cam_exts[camera_name].append(exts)

cam_exts = dict(cam_exts)

In [ ]:
recon = pycolmap.Reconstruction(model_dir)

### Viz

In [ ]:
exo_frame_idx = 0 # changeme
mobile_frame_idx = 0 # changeme
aria_frame_idx = -1 # changeme
inc_mob_aria_in_exo = False # whether we want to visualize mobile and aria points in the exo perspectives

cam_names = ["cam01", "cam02", "cam03", "cam04", "mobile", "aria01"]
all_frames =  [exo_frame_idx] * 4 + [mobile_frame_idx, aria_frame_idx]
assert len(cam_names) == len(all_frames)
centers = get_camera_centers(
    cam_names,
    all_frames,
    cam_exts,
    recon,
    include_mobile_aria_in_exo=inc_mob_aria_in_exo,
)
imgs_paths = [
    (name, cam_exts[name][frame]["image_path"] if frame is not None else None)
    for name, frame in zip(cam_names, all_frames)
]

imgs_by_name = {
    name: (idx, cv2.cvtColor(
        cv2.imread(img_path),
        cv2.COLOR_BGR2RGB,
    ) if img_path is not None else None)
    for idx, (name, img_path) in enumerate(imgs_paths)
}

In [ ]:
draw_all(cam_names, centers, imgs_by_name, n_cols=2)